In [2]:
import pandas as pd 
import numpy as np

In [3]:
# testing
print(f'jello {pd.__version__}')

jello 2.3.0


In [ ]:
# LC 184 - https://leetcode.com/problems/department-highest-salary/description/

"""
Table: Employee

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| id           | int     |
| name         | varchar |
| salary       | int     |
| departmentId | int     |
+--------------+---------+
id is the primary key (column with unique values) for this table.
departmentId is a foreign key (reference columns) of the ID from the Department table.
Each row of this table indicates the ID, name, and salary of an employee. It also contains the ID of their department.
 

Table: Department

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
+-------------+---------+
id is the primary key (column with unique values) for this table. It is guaranteed that department name is not NULL.
Each row of this table indicates the ID of a department and its name.
 

Write a solution to find employees who have the highest salary in each of the departments.

Return the result table in any order.


T-SQL :
WITH max_sal_cte AS (
    SELECT 
        e.name AS Employee,
        salary AS Salary, 
        d.name AS Department,
        ROW_NUMBER() OVER(PARTITION BY departmentId ORDER BY salary DESC) AS rn
    FROM Employee AS e
    INNER JOIN Department AS d       # left also works
    ON e.departmentId = d.id
)
SELECT Employee, Salary, Department 
FROM max_sal_cte
WHERE rn = 1;
"""


employee_data = {
    'id': [1, 2, 3, 4, 5],
    'name': ['Joe', 'Jim', 'Henry', 'Sam', 'Max'],
    'salary': [70000, 90000, 80000, 60000, 90000],
    'departmentId': [1, 1, 2, 2, 1]
}

department_data = {
    'id': [1, 2],
    'name': ['IT', 'Sales']
}

employee_df = pd.DataFrame(employee_data)
department_df = pd.DataFrame(department_data)


def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    merged_df = employee.merge(department, left_on='departmentId', right_on='id', how='inner', suffixes=['_emp', '_dept'])
    print('merged_df \n', merged_df)
    max_salaries = merged_df.groupby('name_dept')['salary'].transform('max')
    print(max_salaries)
    result_df = merged_df[merged_df['salary'] == max_salaries]
    print(result_df)
    final_df = result_df[['name_dept', 'name_emp', 'salary']]
    final_df.columns = ['Departmen', 'Employee', 'Salary']
    
    return final_df


print(department_highest_salary(employee_df, department_df))

merged_df 
    id_emp name_emp  salary  departmentId  id_dept name_dept
0       1      Joe   70000             1        1        IT
1       2      Jim   90000             1        1        IT
2       3    Henry   80000             2        2     Sales
3       4      Sam   60000             2        2     Sales
4       5      Max   90000             1        1        IT
0    90000
1    90000
2    80000
3    80000
4    90000
Name: salary, dtype: int64
   id_emp name_emp  salary  departmentId  id_dept name_dept
1       2      Jim   90000             1        1        IT
2       3    Henry   80000             2        2     Sales
4       5      Max   90000             1        1        IT
  Departmen Employee  Salary
1        IT      Jim   90000
2     Sales    Henry   80000
4        IT      Max   90000


In [ ]:
# LC 117 - https://leetcode.com/problems/nth-highest-salary/description/

"""
Table: Employee

+-------------+------+
| Column Name | Type |
+-------------+------+
| id          | int  |
| salary      | int  |
+-------------+------+
id is the primary key (column with unique values) for this table.
Each row of this table contains information about the salary of an employee.
 

Write a solution to find the nth highest distinct salary from the Employee table. 
If there are less than n distinct salaries, return null.


T SQL

CREATE FUNCTION dbo.getNthHighestSalary (@N INT)
RETURNS INT
AS
BEGIN
    -- Return NULL if N is invalid
    IF @N < 1
        RETURN NULL;

    RETURN (
        SELECT DISTINCT(salary)
        FROM Employee
        ORDER BY salary DESC
        OFFSET (@N - 1) ROWS FETCH NEXT 1 ROWS ONLY
    );
END;
"""


data = [[1, 100], [2, 200], [3, 300]]
employee_df = pd.DataFrame(data, columns=['Id', 'Salary']).astype({'Id':'Int64', 'Salary':'Int64'})

def nth_highest_salary(employee: pd.DataFrame, n: int) -> pd.DataFrame:
    sorted_salary = employee['salary'].sort_values(ascending=False)
    unique_values = sorted_salary.unique()
    
    col_name = f'getNthHighestSalary({n})'
    
    if n < 1 or n > len(unique_values):
        return pd.DataFrame({col_name: [None]})
    else:
        nth_val = int(unique_values[n - 1])
        return pd.DataFrame({col_name: [nth_val]})


nth_highest_salary(employee_df, 2)

200


200

In [ ]:
# LC 570 - https://leetcode.com/problems/managers-with-at-least-5-direct-reports/description/


"""
Table: Employee

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
| department  | varchar |
| managerId   | int     |
+-------------+---------+
id is the primary key (column with unique values) for this table.
Each row of this table indicates the name of an employee, their department, and the id of their manager.
If managerId is null, then the employee does not have a manager.
No employee will be the manager of themself.
 

Write a solution to find managers with at least five direct reports.

Return the result table in any order.


TSQL
Q1
-- select m.name
-- from employee m
-- join employee e
-- on m.id = e.managerId
-- group by m.id, m.name
-- having count(e.managerId) >= 5;

Q2
select m.name as name
from employee m
join (
    select managerId 
    from employee 
    group by managerId
    having count(managerId) >= 5
) as e
on m.id = e.managerId

here, Q2 is faster because it filters the managerId first
and then joins. this gives improved performance on large table 
"""


data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
emp_df = pd.DataFrame(data, columns=['id', 'name', 'department', 'managerId']).astype({'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'})

def find_managers(employee: pd.DataFrame) -> pd.DataFrame:
    df = employee.groupby('managerId')['id'].count().reset_index().rename(columns={'id': 'count'})
    five_reports = df[df['count'] >= 5]['managerId'].values
    return employee[employee['id'].isin(five_reports)][['name']]

print(find_managers(emp_df))

   name
0  John


In [ ]:
# LC 1934 - https://leetcode.com/problems/confirmation-rate/description/?envType=study-plan-v2&envId=top-sql-50

"""
Table: Signups

+----------------+----------+
| Column Name    | Type     |
+----------------+----------+
| user_id        | int      |
| time_stamp     | datetime |
+----------------+----------+
user_id is the column of unique values for this table.
Each row contains information about the signup time for the user with ID user_id.
 

Table: Confirmations

+----------------+----------+
| Column Name    | Type     |
+----------------+----------+
| user_id        | int      |
| time_stamp     | datetime |
| action         | ENUM     |
+----------------+----------+
(user_id, time_stamp) is the primary key (combination of columns with unique values) for this table.
user_id is a foreign key (reference column) to the Signups table.
action is an ENUM (category) of the type ('confirmed', 'timeout')
Each row of this table indicates that the user with ID user_id requested a confirmation message at time_stamp and that confirmation message was either confirmed ('confirmed') or expired without confirming ('timeout').
 

The confirmation rate of a user is the number of 'confirmed' messages divided by the total number of requested confirmation messages. The confirmation rate of a user that did not request any confirmation messages is 0. Round the confirmation rate to two decimal places.

Write a solution to find the confirmation rate of each user.

Return the result table in any order.


T SQL

SELECT 
    s.user_id,
    ROUND(
        CASE
            WHEN COUNT(c.action) = 0 THEN 0
            ELSE SUM(CASE WHEN c.action = 'confirmed' THEN 1 ELSE 0 END) * 1.0 / COUNT(c.action)
        END 
    , 2) AS confirmation_rate 
FROM Signups AS s
LEFT JOIN Confirmations AS c
ON s.user_id = c.user_id
GROUP BY s.user_id

"""


data = [[3, '2020-03-21 10:16:13'], [7, '2020-01-04 13:57:59'], [2, '2020-07-29 23:09:44'], [6, '2020-12-09 10:39:37']]
signups_df = pd.DataFrame(data, columns=['user_id', 'time_stamp']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]'})
data = [[3, '2021-01-06 03:30:46', 'timeout'], [3, '2021-07-14 14:00:00', 'timeout'], [7, '2021-06-12 11:57:29', 'confirmed'], [7, '2021-06-13 12:58:28', 'confirmed'], [7, '2021-06-14 13:59:27', 'confirmed'], [2, '2021-01-22 00:00:00', 'confirmed'], [2, '2021-02-28 23:59:59', 'timeout']]
confirmations_df = pd.DataFrame(data, columns=['user_id', 'time_stamp', 'action']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]', 'action':'object'})


def confirmation_rate(signups: pd.DataFrame, confirmations: pd.DataFrame) -> pd.DataFrame:
    combined_df = pd.merge(signups, confirmations, how='left', on='user_id')
    combined_df['action_code'] = combined_df['action'].map({'confirmed': 1, 'timeout': 0})
    combined_df = combined_df.groupby('user_id')['action_code'].mean().round(2).fillna(0).reset_index()
    combined_df.columns = ['user_id', 'confirmation_rate']
    return combined_df


confirmation_rate(signups_df, confirmations_df)

   user_id        time_stamp_x        time_stamp_y     action  action_code
0        3 2020-03-21 10:16:13 2021-01-06 03:30:46    timeout          0.0
1        3 2020-03-21 10:16:13 2021-07-14 14:00:00    timeout          0.0
2        7 2020-01-04 13:57:59 2021-06-12 11:57:29  confirmed          1.0
3        7 2020-01-04 13:57:59 2021-06-13 12:58:28  confirmed          1.0
4        7 2020-01-04 13:57:59 2021-06-14 13:59:27  confirmed          1.0
5        2 2020-07-29 23:09:44 2021-01-22 00:00:00  confirmed          1.0
6        2 2020-07-29 23:09:44 2021-02-28 23:59:59    timeout          0.0
7        6 2020-12-09 10:39:37                 NaT        NaN          NaN
   user_id  confirmation_rate
0        2                0.5
1        3                0.0
2        6                0.0
3        7                1.0


In [ ]:
# LC 1193 - https://leetcode.com/problems/monthly-transactions-i/description/?envType=study-plan-v2&envId=top-sql-50

"""
Table: Transactions

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| country       | varchar |
| state         | enum    |
| amount        | int     |
| trans_date    | date    |
+---------------+---------+
id is the primary key of this table.
The table has information about incoming transactions.
The state column is an enum of type ["approved", "declined"].
 

Write an SQL query to find for each month and country, the number of transactions and their total amount, 
the number of approved transactions and their total amount.

Return the result table in any order.


T SQL

Q1
-- WITH tx AS (
--   SELECT *,
--     FORMAT(trans_date, 'yyyy-MM') AS month
--   FROM Transactions
-- )
-- SELECT
--   country,
--   month,
--   COUNT(*) AS trans_count,
--   SUM(CASE WHEN state = 'approved' THEN 1 ELSE 0 END) AS approved_count,
--   SUM(amount) AS trans_total_amount,
--   SUM(CASE WHEN state = 'approved' THEN amount ELSE 0 END) AS approved_total_amount
-- FROM tx
-- GROUP BY country, month;


Q2
SELECT 
    country,
    FORMAT(trans_date, 'yyyy-MM') AS month,
    COUNT(*) AS trans_count,
    SUM(IIF(state = 'approved', 1, 0)) AS approved_count,
    SUM(amount) AS trans_total_amount,
    SUM(IIF(state = 'approved', amount, 0)) AS approved_total_amount
FROM Transactions
GROUP BY country, FORMAT(trans_date, 'yyyy-MM')

the only difference is in Q2 I'm using format function twice but in 
Q1 the format is used once inside the cte. both gives same performance in LC.
also case and iif(inline if) are interchangeable
"""


data = [
    [121, 'US', 'approved', 1000, '2018-12-18'], 
    [122, 'US', 'declined', 2000, '2018-12-19'], 
    [123, 'US', 'approved', 2000, '2019-01-01'], 
    [124, 'DE', 'approved', 2000, '2019-01-07']
]
transactions_df = pd.DataFrame(data, columns=['id', 'country', 'state', 'amount', 'trans_date']).astype({'id':'Int64', 'country':'object', 'state':'object', 'amount':'Int64', 'trans_date':'datetime64[ns]'})


def monthly_transactions(transactions: pd.DataFrame) -> pd.DataFrame:
    transactions['month'] = pd.to_datetime(transactions['trans_date']).dt.strftime('%Y-%m')
    transactions['state'] = transactions['state'].str.replace('approved', '1').replace('declined', '0')
    transactions['state'] = transactions['state'].astype(int)
    transactions['approved_total_amount'] = transactions['amount'] * transactions['state']
    resq = transactions.groupby(['month', 'country'], dropna=False).agg({
        'id' : 'count', 
        'state' : 'sum', 
        'amount' : 'sum', 
        'approved_total_amount' : 'sum'}).reset_index()
    return resq.rename(columns={'id' : 'trans_count', 'state' : 'approved_count', 'amount' : 'trans_total_amount'})


print(monthly_transactions(transactions_df))

    id country state  amount trans_date    month
0  121      US     1    1000 2018-12-18  2018-12
1  122      US     0    2000 2018-12-19  2018-12
2  123      US     1    2000 2019-01-01  2019-01
3  124      DE     1    2000 2019-01-07  2019-01
     month country  trans_count  approved_count  trans_total_amount  \
0  2018-12      US            2               1                3000   
1  2019-01      DE            1               1                2000   
2  2019-01      US            1               1                2000   

   approved_total_amount  
0                   1000  
1                   2000  
2                   2000  


In [ ]:
# LC 1174 - https://leetcode.com/problems/immediate-food-delivery-ii/description/?envType=study-plan-v2&envId=top-sql-50


"""

Table: Delivery

+-----------------------------+---------+
| Column Name                 | Type    |
+-----------------------------+---------+
| delivery_id                 | int     |
| customer_id                 | int     |
| order_date                  | date    |
| customer_pref_delivery_date | date    |
+-----------------------------+---------+
delivery_id is the column of unique values of this table.
The table holds information about food delivery to customers that make orders at some date and specify a preferred delivery date (on the same order date or after it).
 

If the customer's preferred delivery date is the same as the order date, then the order is called immediate; otherwise, it is called scheduled.

The first order of a customer is the order with the earliest order date that the customer made. It is guaranteed that a customer has precisely one first order.

Write a solution to find the percentage of immediate orders in the first orders of all customers, rounded to 2 decimal places.


WITH ranked_data AS (
    SELECT 
        *,
        RANK() OVER(PARTITION BY customer_id ORDER BY order_date) AS rnk
    FROM Delivery
)
SELECT
    ROUND(SUM(CASE 
        WHEN order_date = customer_pref_delivery_date THEN 1
        ELSE 0
    END) * 100.0 / COUNT(*), 2) AS immediate_percentage
FROM ranked_data
WHERE rnk = 1
"""


data = [
    [1, 1, '2019-08-01', '2019-08-02'], 
    [2, 2, '2019-08-02', '2019-08-02'], 
    [3, 1, '2019-08-11', '2019-08-12'], 
    [4, 3, '2019-08-24', '2019-08-24'], 
    [5, 3, '2019-08-21', '2019-08-22'], 
    [6, 2, '2019-08-11', '2019-08-13'], 
    [7, 4, '2019-08-09', '2019-08-09']
]
delivery_df = pd.DataFrame(data, columns=['delivery_id', 'customer_id', 'order_date', 'customer_pref_delivery_date']).astype({
    'delivery_id':'Int64', 
    'customer_id':'Int64', 
    'order_date':'datetime64[ns]', 
    'customer_pref_delivery_date':'datetime64[ns]'
})


def immediate_food_delivery(delivery: pd.DataFrame) -> pd.DataFrame:
    first = (
        (delivery.sort_values(['customer_id', 'order_date']))
        .drop_duplicates('customer_id', keep='first')
    )
    first['is_immediate'] = (first['order_date'] == first['customer_pref_delivery_date']).astype(int)
    immediate_mean = round(first['is_immediate'].mean() * 100, 2)
    return pd.DataFrame(data={'immediate_percentage': immediate_mean}, index=[0])

print(immediate_food_delivery(delivery_df))



   immediate_percentage
0                  50.0


In [ ]:
# LC 580 - https://leetcode.com/problems/game-play-analysis-iv/?envType=study-plan-v2&envId=top-sql-50

"""
Table: Activity

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |
+--------------+---------+
(player_id, event_date) is the primary key (combination of columns with unique values) of this table.
This table shows the activity of players of some games.
Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on someday using some device.

Write a solution to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to determine the number of players who logged in on the day immediately following their initial login, and divide it by the number of total players.



SELECT 
    ROUND(COUNT(DISTINCT b.player_id) * 1.0 / 
    COUNT(DISTINCT a.player_id), 2) AS fraction
FROM (
    SELECT player_id, MIN(event_date) AS event_date
    FROM Activity
    GROUP BY player_id
) AS a
LEFT JOIN Activity AS b
ON 
    a.player_id = b.player_id   
    AND DATEADD(day, 1, a.event_date) = b.event_date
"""

data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-03-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity_df = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})


def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    activity = activity.sort_values(by=["player_id", "event_date"])
    activity["first_login"] = activity.groupby("player_id")["event_date"].transform("first")
    activity["next_login"] = activity.groupby("player_id")["event_date"].shift(-1)
    activity["is_consecutive"] = (activity["next_login"] == activity["first_login"] + pd.Timedelta(days=1))
    consecutive_players = activity[activity['is_consecutive']]['player_id'].nunique()
    total_players = activity['player_id'].nunique()
    fraction = round(consecutive_players / total_players, 2)
    return pd.DataFrame(data={'fraction': fraction}, index=[0])

print(gameplay_analysis(activity_df))

   fraction
0      0.33


In [ ]:
# LC 185 - https://leetcode.com/problems/department-top-three-salaries/description/?envType=study-plan-v2&envId=top-sql-50


"""
Table: Employee

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| id           | int     |
| name         | varchar |
| salary       | int     |
| departmentId | int     |
+--------------+---------+
id is the primary key (column with unique values) for this table.
departmentId is a foreign key (reference column) of the ID from the Department table.
Each row of this table indicates the ID, name, and salary of an employee. It also contains the ID of their department.



Q1
WITH sal_rnk AS (
    SELECT 
        d.name AS Department,
        e.name AS Employee,
        e.salary AS Salary,
        DENSE_RANK() OVER(PARTITION BY d.name ORDER BY e.salary DESC) AS rnk
    FROM Employee AS e
    INNER JOIN Department AS d
    ON e.departmentId = d.id
) 
SELECT Department, Employee, Salary
FROM sal_rnk 
WHERE rnk < 4


Q2
SELECT 
    e.name AS Employee,
    d.name AS Department,
    e.salary AS Salary
FROM (
    SELECT 
        departmentId, 
        name, 
        salary,
        DENSE_RANK() OVER(PARTITION BY departmentId ORDER BY salary DESC) AS rnk
    FROM Employee
) AS e
JOIN Department AS d
ON e.departmentId = d.id
WHERE rnk <= 3;
"""


print('asdf')


asdf


In [ ]:
# LC 602 - https://leetcode.com/problems/friend-requests-ii-who-has-the-most-friends/description/?envType=study-plan-v2&envId=top-sql-50

"""
Table: RequestAccepted

+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| requester_id   | int     |
| accepter_id    | int     |
| accept_date    | date    |
+----------------+---------+
(requester_id, accepter_id) is the primary key (combination of columns with unique values) for this table.
This table contains the ID of the user who sent the request, the ID of the user who received the request, and the date when the request was accepted.
 

Write a solution to find the people who have the most friends and the most friends number.

The test cases are generated so that only one person has the most friends.

SELECT TOP 1
    id,
    count(id) AS num
FROM (
    SELECT requester_id AS id
    FROM RequestAccepted
    UNION ALL
    SELECT accepter_id AS id
    FROM RequestAccepted
) AS sub
GROUP BY id
ORDER BY count(id) DESC

"""













In [32]:
# LC 1341 - https://leetcode.com/problems/movie-rating/?envType=study-plan-v2&envId=top-sql-50

"""
Table: Movies

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| movie_id      | int     |
| title         | varchar |
+---------------+---------+
movie_id is the primary key (column with unique values) for this table.
title is the name of the movie.
 

Table: Users

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | int     |
| name          | varchar |
+---------------+---------+
user_id is the primary key (column with unique values) for this table.
The column 'name' has unique values.
Table: MovieRating

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| movie_id      | int     |
| user_id       | int     |
| rating        | int     |
| created_at    | date    |
+---------------+---------+
(movie_id, user_id) is the primary key (column with unique values) for this table.
This table contains the rating of a movie by a user in their review.
created_at is the user's review date. 
 

Write a solution to:

Find the name of the user who has rated the greatest number of movies. 
    In case of a tie, return the lexicographically smaller user name.
Find the movie name with the highest average rating in February 2020. 
    In case of a tie, return the lexicographically smaller movie name.

lexicographically - alphabetical order

T SQL

/* Write your T-SQL query statement below */
select results
from(
    select top 1 name as results
    from MovieRating m inner join users u 
    on u.user_id =m.user_id
    group by  u.user_id,name 
    order by count(*) desc ,[name] asc
) a 
union all
select results
from (
    select top 1 title as results
    from MovieRating mr inner join movies m 
    on m.movie_id =mr.movie_id
    where month(created_at)=2 and year(created_at)=2020 
    group by m.movie_id ,title
    order by avg(rating*1.00) desc ,title asc
) b
"""

data = [[1, 'Avengers'], [2, 'Frozen 2'], [3, 'Joker']]
movies_df = pd.DataFrame(data, columns=['movie_id', 'title']).astype({'movie_id':'Int64', 'title':'object'})
data = [[1, 'Daniel'], [2, 'Monica'], [3, 'Maria'], [4, 'James']]
users_df = pd.DataFrame(data, columns=['user_id', 'name']).astype({'user_id':'Int64', 'name':'object'})
data = [
    [1, 1, 3, '2020-01-12'], 
    [1, 2, 4, '2020-02-11'], 
    [1, 3, 2, '2020-02-12'], 
    [1, 4, 1, '2020-01-01'], 
    [2, 1, 5, '2020-02-17'], 
    [2, 2, 2, '2020-02-01'], 
    [2, 3, 2, '2020-03-01'], 
    [3, 1, 3, '2020-02-22'], 
    [3, 2, 4, '2020-02-25']
]
movie_rating_df = pd.DataFrame(data, columns=['movie_id', 'user_id', 'rating', 'created_at']).astype({'movie_id':'Int64', 'user_id':'Int64', 'rating':'Int64', 'created_at':'datetime64[ns]'})



def movie_rating(movies: pd.DataFrame, users: pd.DataFrame, movie_rating: pd.DataFrame) -> pd.DataFrame:
    umr_df = pd.merge(users, movie_rating, on='user_id')
    user_with_most_rating = (
        umr_df.groupby(['user_id', 'name'])
        .size()
        .reset_index(name='review_count')
        .sort_values(['review_count', 'name'], ascending=[False, True])
        .iloc[0]['name']
    )

    mmr_df = pd.merge(movies, movie_rating, on='movie_id')
    mmr_df['created_at'] = pd.to_datetime(mmr_df['created_at'])
    mmr_df = mmr_df[
        (mmr_df['created_at'].dt.year == 2020) &
        (mmr_df['created_at'].dt.month == 2)
    ]

    movie_with_max_avg_rating_in_feb2020 = (
        mmr_df.groupby(['movie_id', 'title'])['rating']
        .mean()
        .reset_index(name='avg_rating')
        .sort_values(['avg_rating', 'title'], ascending=[False, False])
        .iloc[0]['title']
    )

    return pd.DataFrame({
        'results': [user_with_most_rating, movie_with_max_avg_rating_in_feb2020]
    })

print(movie_rating(movies_df, users_df, movie_rating_df))

# movie_rating_df, movies_df

  results
0  Daniel
1   Joker


In [ ]:
# LC 1070 - https://leetcode.com/problems/product-sales-analysis-iii/?envType=study-plan-v2&envId=top-sql-50

""" 
Table: Sales

+-------------+-------+
| Column Name | Type  |
+-------------+-------+
| sale_id     | int   |
| product_id  | int   |
| year        | int   |
| quantity    | int   |
| price       | int   |
+-------------+-------+
(sale_id, year) is the primary key (combination of columns with unique values) of this table.
product_id is a foreign key (reference column) to Product table.
Each row records a sale of a product in a given year.
A product may have multiple sales entries in the same year.
Note that the per-unit price.

Write a solution to find all sales that occurred in the first year each product was sold.

For each product_id, identify the earliest year it appears in the Sales table.

Return all sales entries for that product in that year.

Return a table with the following columns: product_id, first_year, quantity, and price.
Return the result in any order.


WITH CTE AS (
    SELECT product_id, MIN(year) AS minyear FROM Sales 
    GROUP BY product_id 
)

SELECT s.product_id, s.year AS first_year, s.quantity, s.price 
FROM Sales s
INNER JOIN CTE 
ON cte.product_id = s.product_id  
    AND s.year = cte.minyear; 
"""



data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales_df = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype({'sale_id':'Int64', 'product_id':'Int64', 'year':'Int64', 'quantity':'Int64', 'price':'Int64'})

def sales_analysis(sales: pd.DataFrame) -> pd.DataFrame:
    prod_year_gp = sales.groupby('product_id')['year'].min().reset_index(name='first_year')
    results = sales.merge(prod_year_gp, how='inner', left_on=['product_id', 'year'], right_on=['product_id', 'first_year'])
    return results[['product_id', 'first_year', 'quantity', 'price']]

print(sales_analysis(sales_df))


   product_id  first_year  quantity  price
0         100        2008        10   5000
1         200        2011        15   9000


In [ ]:
# LC 1045 - https://leetcode.com/problems/customers-who-bought-all-products/description/?envType=study-plan-v2&envId=top-sql-50


""" 
Table: Customer

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| customer_id | int     |
| product_key | int     |
+-------------+---------+
This table may contain duplicates rows. 
customer_id is not NULL.
product_key is a foreign key (reference column) to Product table.
 

Table: Product

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_key | int     |
+-------------+---------+
product_key is the primary key (column with unique values) for this table.
 

Write a solution to report the customer ids from the Customer table that bought all the products in the Product table.

Return the result table in any order.



with cl_cust as (
    select distinct customer_id, product_key
    from Customer
)
select customer_id
from cl_cust as c
inner join Product as p
on c.product_key = p.product_key
group by customer_id
having count(*) = (select count(distinct product_key) from Product)


"""

In [12]:
# LC 180 - https://leetcode.com/problems/consecutive-numbers/description/?envType=study-plan-v2&envId=top-sql-50


""" 
Table: Logs

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| num         | varchar |
+-------------+---------+
In SQL, id is the primary key for this table.
id is an autoincrement column starting from 1.
 

Find all numbers that appear at least three times consecutively.

Return the result table in any order.

T SQL:
select distinct a.num as ConsecutiveNums 
from Logs a
join Logs b 
on b.id = a.id + 1 and b.num = a.num
join Logs c 
on c.id = a.id + 2 and c.num = a.num
"""

data = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2]]
logs_df = pd.DataFrame(data, columns=['id', 'num']).astype({'id':'Int64', 'num':'Int64'})


def consecutive_numbers(logs: pd.DataFrame) -> pd.DataFrame:
    logs.sort_values('id', inplace=True)
    logs['prev'] = logs.num.shift(1)
    logs['next'] = logs.num.shift(-1)
    results = logs[(logs['num'] == logs['prev']) & (logs['num'] == logs['next'])]['num'].unique()
    result_df = pd.DataFrame(results, columns=['ConsecutiveNums'])
    return result_df

print(consecutive_numbers(logs_df))


   ConsecutiveNums
0                1


In [8]:
# LC 1164 - https://leetcode.com/problems/product-price-at-a-given-date/description/?envType=study-plan-v2&envId=top-sql-50

""" 
Table: Products

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| new_price     | int     |
| change_date   | date    |
+---------------+---------+
(product_id, change_date) is the primary key (combination of columns with unique values) of this table.
Each row of this table indicates that the price of some product was changed to a new price at some date.
Initially, all products have price 10.

Write a solution to find the prices of all products on the date 2019-08-16.

Return the result table in any order.


T SQL

select 
    product_id,
    first_value(new_price) over(partition by product_id order by change_date desc) as price
from Products
where change_date <= '2019-08-16'
union 
select 
    product_id,
    10 as price
from Products 
group by product_id
having min(change_date) > '2019-08-16'



-- with latest_date as (
--     select 
--         product_id, max(change_date) as change_date 
--     from Products 
--     where change_date <= '2019-08-16'
--     group by product_id
-- ), prod_price as (
--     select p.product_id, p.new_price as price
--     from Products as p
--     inner join latest_date as ld
--     on 
--         p.product_id = ld.product_id and
--         p.change_date = ld.change_date
-- )
-- select 
--     distinct p.product_id,
--     coalesce(pp.price, 10) as price 
-- from Products as p
-- left join prod_price as pp 
-- on p.product_id = pp.product_id
"""


data = [[1, 20, '2019-08-14'], [2, 50, '2019-08-14'], [1, 30, '2019-08-15'], [1, 35, '2019-08-16'], [2, 65, '2019-08-17'], [3, 20, '2019-08-18']]
products_df = pd.DataFrame(data, columns=['product_id', 'new_price', 'change_date']).astype({'product_id':'Int64', 'new_price':'Int64', 'change_date':'datetime64[ns]'})


def price_at_given_date(products: pd.DataFrame) -> pd.DataFrame:
    target_date = pd.to_datetime('2019-08-16')
    valid_prods = products[products['change_date'] <= target_date].sort_values(['product_id', 'change_date'])
    latest_prods = valid_prods.groupby('product_id', as_index=False).last()[['product_id', 'new_price']]
    all_prods = products['product_id'].unique()
    result = pd.DataFrame({'product_id': all_prods}).merge(latest_prods, on='product_id', how='left').fillna({'new_price': 10})
    result.rename(columns={'new_price': 'price'}, inplace=True)
    return result


print(price_at_given_date(products_df))



   product_id  price
0           1     35
1           2     50
2           3     10


In [ ]:
# LC 1204 - https://leetcode.com/problems/last-person-to-fit-in-the-bus/description/?envType=study-plan-v2&envId=top-sql-50


""" 
Table: Queue

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| person_id   | int     |
| person_name | varchar |
| weight      | int     |
| turn        | int     |
+-------------+---------+
person_id column contains unique values.
This table has the information about all people waiting for a bus.
The person_id and turn columns will contain all numbers from 1 to n, where n is the number of rows in the table.
turn determines the order of which the people will board the bus, 
where turn = 1 denotes the first person to board and turn=n denotes the last person to board.
weight is the weight of the person in kilograms.
 

There is a queue of people waiting to board a bus. 
However, the bus has a weight limit of 1000 kilograms, so there may be some people who cannot board.

Write a solution to find the person_name of the last person that can fit on the bus without exceeding the weight limit. 
The test cases are generated such that the first person does not exceed the weight limit.

Note that only one person can board the bus at any given turn.

The result format is in the following example.

with r_s as (
    select 
        *,
        sum(weight) over(order by turn) as running_w,
        max(turn) over(order by (select Null)) as max_turn
    from Queue
)
select top 1 person_name
from r_s
where running_w <= 1000 or turn = max_turn
order by 
    case
        when running_w <= 1000 then 0
        else 1
    end,
    turn desc

"""







In [1]:
# LC 1907 - https://leetcode.com/problems/count-salary-categories/?envType=study-plan-v2&envId=top-sql-50

""" 
Table: Accounts

+-------------+------+
| Column Name | Type |
+-------------+------+
| account_id  | int  |
| income      | int  |
+-------------+------+
account_id is the primary key (column with unique values) for this table.
Each row contains information about the monthly income for one bank account.
 

Write a solution to calculate the number of bank accounts for each salary category. The salary categories are:

"Low Salary": All the salaries strictly less than $20000.
"Average Salary": All the salaries in the inclusive range [$20000, $50000].
"High Salary": All the salaries strictly greater than $50000.
The result table must contain all three categories. If there are no accounts in a category, return 0.

Return the result table in any order.

T SQL
select 'Low Salary' As category, sum(iif(income < 20000, 1, 0)) as accounts_count
from Accounts

union 

select 'Average Salary', sum(iif(income between 20000 and 50000, 1, 0))
from Accounts 

union 

select 'High Salary', sum(iif(income > 50000, 1, 0))
from Accounts

"""

' \nTable: Accounts\n\n+-------------+------+\n| Column Name | Type |\n+-------------+------+\n| account_id  | int  |\n| income      | int  |\n+-------------+------+\naccount_id is the primary key (column with unique values) for this table.\nEach row contains information about the monthly income for one bank account.\n\n\nWrite a solution to calculate the number of bank accounts for each salary category. The salary categories are:\n\n"Low Salary": All the salaries strictly less than $20000.\n"Average Salary": All the salaries in the inclusive range [$20000, $50000].\n"High Salary": All the salaries strictly greater than $50000.\nThe result table must contain all three categories. If there are no accounts in a category, return 0.\n\nReturn the result table in any order.\n\nT SQL\nselect \'Low Salary\' As category, sum(iif(income < 20000, 1, 0)) as accounts_count\nfrom Accounts\n\nunion \n\nselect \'Average Salary\', sum(iif(income between 20000 and 50000, 1, 0))\nfrom Accounts \n\nunion

In [ ]:
# LC 626 - https://leetcode.com/problems/exchange-seats/description/?envType=study-plan-v2&envId=top-sql-50

""" 
Table: Seat

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| student     | varchar |
+-------------+---------+
id is the primary key (unique value) column for this table.
Each row of this table indicates the name and the ID of a student.
The ID sequence always starts from 1 and increments continuously.
 

Write a solution to swap the seat id of every two consecutive students. If the number of students is odd, the id of the last student is not swapped.

Return the result table ordered by id in ascending order.

T SQL 

SELECT 
    CASE 
        WHEN id % 2 = 1 AND id + 1 <= (SELECT MAX(id) FROM Seat)
            THEN id + 1
        WHEN id % 2 = 0 THEN id - 1
        ELSE id
    END AS id,
    student
FROM Seat
ORDER BY id;


Q2 - leetcode says this is faster
select 
    main.id, 
    case 
        when id % 2 = 0 then 
            lag(student, 1, student) over(order by (select null))
        else 
            lead(student, 1, student) over(order by (select null))
    end as student
from Seat as main


"""



' \nTable: Seat\n\n+-------------+---------+\n| Column Name | Type    |\n+-------------+---------+\n| id          | int     |\n| student     | varchar |\n+-------------+---------+\nid is the primary key (unique value) column for this table.\nEach row of this table indicates the name and the ID of a student.\nThe ID sequence always starts from 1 and increments continuously.\n\n\nWrite a solution to swap the seat id of every two consecutive students. If the number of students is odd, the id of the last student is not swapped.\n\nReturn the result table ordered by id in ascending order.\n\nT SQL \n\nSELECT \n    CASE \n        WHEN id % 2 = 1 AND id + 1 <= (SELECT MAX(id) FROM Seat)\n            THEN id + 1\n        WHEN id % 2 = 0 THEN id - 1\n        ELSE id\n    END AS id,\n    student\nFROM Seat\nORDER BY id;\n'

In [ ]:
# LC 1321 - https://leetcode.com/problems/restaurant-growth/description/?envType=study-plan-v2&envId=top-sql-50  

""" 
Table: Customer

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| customer_id   | int     |
| name          | varchar |
| visited_on    | date    |
| amount        | int     |
+---------------+---------+
In SQL,(customer_id, visited_on) is the primary key for this table.
This table contains data about customer transactions in a restaurant.
visited_on is the date on which the customer with ID (customer_id) has visited the restaurant.
amount is the total paid by a customer.
 

You are the restaurant owner and you want to analyze a possible expansion (there will be at least one customer every day).

Compute the moving average of how much the customer paid in a seven days window (i.e., current day + 6 days before). average_amount should be rounded to two decimal places.

Return the result table ordered by visited_on in ascending order.


WITH moving_sum_7_window AS (
    SELECT 
        visited_on,
        SUM(SUM(amount)) OVER(
            ORDER BY visited_on
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ) AS amount
    FROM Customer
    GROUP BY visited_on 
) 
SELECT 
    visited_on,
    amount,
    ROUND((amount / 7.0), 2) AS average_amount 
FROM moving_sum_7_window
ORDER BY visited_on
OFFSET 6 ROWS;

"""